In [22]:
import csv
import time
import json
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from selenium.webdriver.common.action_chains import ActionChains


In [28]:
with open('./科研机构失败清单.json', encoding='utf-8') as f:
  institu_list = json.load(f)
with open('./data/长三角普通高等学校名单.json', encoding='utf-8') as f:
  school_list = json.load(f)

In [3]:
institu_list[0]

{'科研机构名称': '江苏省环境科学研究院', '研究领域': '环境科学', '省': '江苏省', '市': '南京市', '县/区': '鼓楼区'}

In [29]:
def open_page(driver, institu_name):
  # 打开知网高级检索页面
  driver.get("https://kns.cnki.net/kns8/AdvSearch?dbprefix=CFLS&&crossDbcodes=CJFQ%2CCDMD%2CCIPD%2CCCND%2CCISD%2CSNAD%2CBDZK%2CCCJD%2CCCVD%2CCJFN")
  # 点击学术期刊栏目
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div[3]/div[1]/div/ul[1]/li[1]/a/span'))).click()
  # 选择作者单位
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[1]/div[1]/span'))).click()
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[1]/div[2]/ul[2]/li[5]/a'))).click()
  time.sleep(3)
  driver.find_element_by_xpath(
      '//*[@id = "gradetxt"]/dd[2]/div[2]/input').send_keys(institu_name)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[2]/div/span'))).click()
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '//*[@id="gradetxt"]/dd[2]/div[2]/div[2]/ul/li[1]/a'))).click()
  time.sleep(3)
  # 选择年份
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/div/input'))).click()
  time.sleep(3)
  time0_element = driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/ul')
  driver.execute_script(
      "arguments[0].scrollTop = arguments[0].scrollHeight", time0_element)
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[1]/ul/li[45]/a'))).click()
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div/input'))).click()
  time.sleep(3)
  WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/ul/li[3]/a'))).click()
  time.sleep(3)

  # 点击检索
  WebDriverWait(driver, 100).until(
      EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/div/div[1]/div[1]/div[2]/div[2]/input'))).click()
  time.sleep(3)
  # 获取中文总文献数
  ch_num = WebDriverWait(driver, 100).until(EC.presence_of_element_located(
      (By.XPATH, '''//*[@id="countPageDiv"]/span[1]/em'''))).text
  ch_num = int(ch_num.replace(",", ''))
  return ch_num
#   # 获取外文总文献数
#   WebDriverWait(driver, 100).until(
#       EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[1]/div/div/div/a[2]'))).click()
#   time.sleep(3)
#   en_num = WebDriverWait(driver, 100).until(EC.presence_of_element_located(
#       (By.XPATH, '''//*[@id="countPageDiv"]/span[1]/em'''))).text
#   en_num = int(en_num.replace(",", ''))
#   return [ch_num, en_num]



In [30]:
if __name__ == "__main__":
    # get直接返回，不再等待界面加载完成
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["pageLoadStrategy"] = "none"

    # 设置谷歌驱动器的环境
    options = webdriver.ChromeOptions()
    # 设置chrome不加载图片，提高速度
    options.add_experimental_option(
        "prefs", {"profile.managed_default_content_settings.images": 2})
    # # 设置不显示窗口
    # options.add_argument('--headless')
    # 创建一个谷歌驱动器
    driver = webdriver.Chrome(options=options)
    # 爬取每个机构发表的中英文论文期刊数
    fail_institu_list = []
    fail_institu_list = []
    success_num = 0
    fail_num = 0
    for institu in institu_list:
        try:
            temp_num = open_page(driver, institu['科研机构名称'])
            institu['中文文献数'] = temp_num
            # institu['英文文献数'] = temp_num[1]
            success_num += 1
        except Exception as e:
            fail_num += 1
            print('fail: ', fail_num, '\t success: ', success_num)
            print('fail_name: ', institu)
            fail_institu_list.append(institu)

    with open('./长三角普通高等学校知网论文数.json', 'w', encoding='utf-8') as file:
        json.dump(institu_list, file, ensure_ascii=False)
    with open('./失败清单.json', 'w', encoding='utf-8') as file:
        json.dump(fail_institu_list, file, ensure_ascii=False)
    # 关闭浏览器
    driver.close()


fail:  1 	 success:  0
fail_name:  {'科研机构名称': '江苏省环境科学研究院', '研究领域': '环境科学', '省': '江苏省', '市': '南京市', '县': {'区': '鼓楼区'}}
fail:  2 	 success:  0
fail_name:  {'科研机构名称': '江苏省老年医学研究所', '研究领域': '老年医学', '省': '江苏省', '市': '南京市', '县': {'区': '鼓楼区'}}
fail:  3 	 success:  0
fail_name:  {'科研机构名称': '江苏省宏图电子综合研究所有限公司', '研究领域': '电子综合研究', '省': '江苏省', '市': '南京市', '县': {'区': '鼓楼区'}}
fail:  4 	 success:  0
fail_name:  {'科研机构名称': '江苏省化工信息中心有限公司', '研究领域': '化工信息', '省': '江苏省', '市': '南京市', '县': {'区': '鼓楼区'}}
fail:  5 	 success:  0
fail_name:  {'科研机构名称': '江苏省粮食科学研究设计院有限公司', '研究领域': '粮食科学', '省': '江苏省', '市': '南京市', '县': {'区': '建邺区'}}
fail:  6 	 success:  0
fail_name:  {'科研机构名称': '江苏省轻工业科学研究设计院有限公司', '研究领域': '轻工业', '省': '江苏省', '市': '南京市', '县': {'区': '建邺区'}}
fail:  7 	 success:  0
fail_name:  {'科研机构名称': '江苏省中医临床研究院', '研究领域': '中医', '省': '江苏省', '市': '南京市', '县': {'区': '秦淮区'}}
fail:  8 	 success:  0
fail_name:  {'科研机构名称': '江苏省临床医学研究院', '研究领域': '临床医学', '省': '江苏省', '市': '南京市', '县': {'区': '秦淮区'}}
fail:  9 	 success:  0
fail

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.127)


In [11]:
with open('./长三角普通高等学校知网论文数.json', 'w', encoding='utf-8') as file:
        json.dump(school_list, file, ensure_ascii=False)
with open('./失败清单.json', 'w', encoding='utf-8') as file:
        json.dump(fail_school_list, file, ensure_ascii=False)


In [15]:
school_list[0]['中文文献数']


138

In [21]:
result = {}
for school in school_list:
  if '中文文献数' not in school:
    continue
  if school['城市'] in result:
    result[school['城市']] += school['中文文献数']
  else:
    result[school['城市']] = school['中文文献数']

import csv

# 创建 CSV 文件并写入数据
with open('result.csv', 'w', encoding='gbk', newline='') as f:
    writer = csv.writer(f)

    # 写入 CSV 文件的标题行
    writer.writerow(['城市', '知网中文文献数'])

    # 遍历字典的列表，并将每个字典的数据写入 CSV 文件中
    for item in result.keys():
        row = [item, result[item]]
        writer.writerow(row)


In [32]:
import json 

with open('./长三角科研机构知网论文数.json', encoding='utf-8') as file:
  data = json.load(file)

for i in data:
  if '中文文献数' not in i:
    i['中文文献数'] = 0

with open('./长三角科研机构知网论文数.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False)
